# Mutation Pipeline

In [18]:
from __future__ import print_function
import os.path
import pandas as pd
import gzip
import sys
import numpy as np

sys.path.insert(0, '..')

from src.CCLE_postp_function import *
from JKBio import Datanalytics as da 
from JKBio import TerraFunction as terra
from JKBio import Helper as h
from JKBio.helper.google_sheet import GSheet
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm

from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier

from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict
from IPython.display import Image,display



%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


Loading BokehJS ...

/home/jeremie/miniconda3/lib/python3.7/site-packages/oauth2client/_helpers.py:255: UserWarning: Cannot access /home/jeremie/.storage.json: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))


InvalidClientSecretsError: ('Error opening file', '/home/jeremie/.client_secret.json', 'No such file or directory', 2)

## boot up

we are instanciating all the parameters needed for this pipeline to run

In [2]:
samplesetname = "20Q4"
prevname="20Q3"
prevversion = 24
prevprevname ='20Q2'
prevprevversion= 22

workspace1="broad-genomics-delivery/Getz_IBM_CellLines_Exomes"
workspace2="broad-firecloud-ccle/CCLE_DepMap_WES"
workspace3="broad-genomics-delivery/CCLE_DepMap_WES"

workspace6="terra-broad-cancer-prod/CCLE_DepMap_WES"

refworkspace="broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline"

rnaworkspace="broad-firecloud-ccle/DepMap_hg38_RNAseq"

source1="ibm"
source2="ccle"
source3="ccle"
source6="ccle"
source7="ibm"

refsheet_url = "https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE"
refsheet_id = "555466897"
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"

release = samplesetname


In [3]:
wm1 = dm.WorkspaceManager(workspace1)
wm2 = dm.WorkspaceManager(workspace2)
wm3 = dm.WorkspaceManager(workspace3)

wm6 = dm.WorkspaceManager(workspace6)

refwm = dm.WorkspaceManager(refworkspace)

In [4]:
extract_to_change = {'from_arxspan_id': 'participant'}

In [5]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame().set_index("cds_sample_id")

In [6]:
refsheet = GSheet('/home/jeremie/'+mystorage_id[2:], refsheet_url, refsheet_id)

NameError: name 'STORAGE' is not defined

## Adding new data

We are looking for new samples in a range of workspaces.

They are quite messy and might contains duplicates, contain broken file paths...

- We are thus looking at the bam files one by one and comparing them with our own bams. 
- We remove broken files, duplicates and add new version of a cell line's bam if we find some.

In [250]:
# we will be missing "primary disease","sm_id", "cellosaurus_id", "gender, "age", "primary_site", "primary_disease", "subtype", "subsubtype", "origin", "comments"
#when SMid: match== 
samples, pairs, noarxspan = GetNewCellLinesFromWorkspaces(refworkspace, stype='wes', refurl=refsheet_url, wmfroms = [workspace1, workspace2, workspace6], sources=[source1, source2, source6], match=['ACH-','CDS-'], participantslicepos=10, accept_unknowntypes=True, extract=extract_to_change, recomputedate=True)

refsamples is overrided by a refurl
[23/Sep/2020 23:00:56] WARNING - file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/home/jeremie/miniconda3/lib/python3.7/site-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jeremie/miniconda3/lib/python3.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jeremie/miniconda3/lib/python3.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, 

/home/jeremie/ccle_processing/src/CCLE_postp_function.py:238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  [i for i in samples[extract["bam"]] if type(i) is str and str(i) != 'NA'], "-L", 200)]


listing files in gs


/home/jeremie/ccle_processing/src/CCLE_postp_function.py:240: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples[extract['size']] = [gcp.extractSize(i)[1] for i in gcp.lsFiles(samples[extract['bam']].tolist(), '-al', 200)]


/home/jeremie/ccle_processing/src/CCLE_postp_function.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples[extract["release_date"]] = h.getBamDate(samples[extract["bam"]])
/home/jeremie/ccle_processing/src/CCLE_postp_function.py:250: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples[extract['release_date']] = list(h.datetoint(samples[extract['release_date']].values))
/home/jeremie/ccle_processing/src/CCLE_postp_function.py:253: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

listing files in gs
listing files in gs
we found and removed 0 samples which did not match our id names: ['ACH-', 'CDS-']
found 103 likely replicate
listing files in gs
Len of samples before removal: 103
Dups from this workspace has len 0:
 []
Len of samples after removal: 103

The shape of the sample tsv from <dalmatian.wmanager.WorkspaceManager broad-firecloud-ccle/CCLE_DepMap_WES>: (288, 224)
Identifying any true duplicates by checking file hashes (this runs for each data source)...
This step can take a while as we need to use gsutil to check the size of each potential duplicate...
listing files in gs
These 1 bam file path do not exist: {''}
listing files in gs
listing files in gs
listing files in gs
listing files in gs
we found and removed 0 samples which did not match our id names: ['ACH-', 'CDS-']
found 17 likely replicate
listing files in gs
Len of samples before removal: 17
Dups from this workspace has len 0:
 []
Len of samples after removal: 17

The shape of the sample tsv fro

In [251]:
noarxspan = noarxspan.sort_values(by='stripped_cell_line_name')

In [161]:
noarxspan.to_csv('temp/noarxspan_wes_'+release+'.csv')

### finding back arxspan

In [252]:
noarxspan['arxspan_id'] = ["ACH-001001", "ACH-000520", "ACH-000740", "ACH-000283", "ACH-000757", "ACH-001328", "ACH-000157", "ACH-000557", "ACH-000593", "ACH-001454", "ACH-001456", "ACH-001458", "ACH-001459", "ACH-001460", "ACH-001461", "ACH-000511", "ACH-000025", "ACH-000458", "ACH-001339", "ACH-000662", "ACH-000695", "ACH-000278", "ACH-000123", "ACH-000608", "ACH-001061", "ACH-000698", "ACH-000877", "ACH-001496", "ACH-001497", "ACH-000487", "ACH-001067", "ACH-001500", "ACH-000047", "ACH-001345", "ACH-000840", "ACH-000868", "ACH-000901", "ACH-000143", "ACH-000150", "ACH-000339", "ACH-000872", "ACH-000029", "ACH-000941", "ACH-000946", "ACH-000954", "ACH-000004", "ACH-000005", "ACH-000393", "ACH-001522", "ACH-000274", "ACH-001523", "ACH-000509", "ACH-000672", "ACH-000310", "ACH-000577", "ACH-000237", "ACH-000993", "ACH-001098", "ACH-000167", "ACH-000419", "ACH-000028", "ACH-000823", "ACH-001113", "ACH-000596", "ACH-000591", "ACH-000634", "ACH-000673", "ACH-000676", "ACH-000128", "ACH-000215", "ACH-000760", "ACH-000007", "ACH-000152", "ACH-001550", "ACH-001551", "ACH-001552", "ACH-000019", "ACH-000884", "ACH-001554", "ACH-001555", "ACH-001556", "ACH-001557", "ACH-001558", "ACH-001559", "ACH-001560", "ACH-001561", "ACH-001562", "ACH-000758", "ACH-001563", "ACH-001566", "ACH-001567", "ACH-001568", "ACH-001569", "ACH-001570", "ACH-001129", "ACH-000866", "ACH-000514", "ACH-000921", "ACH-000434", "ACH-000010", "ACH-000912", "ACH-000700", "ACH-000251", "ACH-001075", "ACH-000337", "ACH-000837", "ACH-000800", "ACH-000767", "ACH-000378", "ACH-000200", "ACH-001368", "ACH-000436", "ACH-000247", "ACH-000544", "ACH-000296", "ACH-001373", "ACH-001151", "ACH-000022", "ACH-000606", "ACH-000960", "ACH-000791", "ACH-000774", "ACH-000261", "ACH-000398", "ACH-000473", "ACH-001386", "ACH-001645", "ACH-000887", "ACH-000655", "ACH-000490", "ACH-001190", "ACH-000312", "ACH-001194", "ACH-000017", "ACH-001654", "ACH-000127", "ACH-000302", "ACH-000461", "ACH-000466", "ACH-000736", "ACH-000898", "ACH-000537", "ACH-000460", "ACH-000280", "ACH-000316", "ACH-001390", "ACH-001391", "ACH-001394", "ACH-000122", "ACH-000677", "ACH-000820", "ACH-001402", "ACH-000452", "ACH-001210", "ACH-000036", "ACH-000262", "ACH-000304", "ACH-001709", "ACH-000836",]

In [253]:
noarxspan.loc[noarxspan[noarxspan['stripped_cell_line_name']=="SUM299PE1"].index,"stripped_cell_line_name"] = "SUM299PE"

In [254]:
noarxspan = resolveFromWorkspace(noarxspan, refsamples = ccle_refsamples[ccle_refsamples['datatype'] == 'wes'], match = ['ACH','CDS'], participantslicepos = 10, accept_unknowntypes = True, extract = extract_to_change)

we found and removed 0 samples which did not match our id names: ['ACH', 'CDS']
found 159 likely replicate
listing files in gs
Len of samples before removal: 159
Dups from this workspace has len 0:
 []
Len of samples after removal: 159


In [255]:
#assess any potential issues
set(noarxspan.arxspan_id) & set(samples.arxspan_id)

set()

In [256]:
samples = pd.concat([samples, noarxspan], sort=False)

In [257]:
samples = assessAllSamples(samples, ccle_refsamples, stype='wes', rename={}, extract={})

we had 0 duplicates in the release buckets


In [258]:
set(pairs.control_sample)

{'nan'}

In [184]:
#TODO: manage the match normals in noarxspan samples

## getting the addtional data and writing it here in the right order 'as shown above'
- use the stripped_cell_line_name to find the samples on https://docs.google.com/spreadsheets/d/1uqCOos-T9EMQU7y2ZUw4Nm84opU5fIT1y7jet1vnScE/edit#gid=356471436. 
- Make sure that we don't have duplicate cell lines in there. Otherwise, use the duplicate renaming function
- copy Primary Site, Primary Disease, Subtype, Comments, Disease Sub-subtype, if they exist. (sometimes subtype and subsubtype are the same.. don't use subsubtype then.
- look for the cell line in cellosaurus, you might need to use one of the aliases given in master depmap pv..
- copy  cellosaurus_id gender age info or write 'U' if they don't exist. 'can be a number or {Embryonic, Children, Adult, Fetus, U} 
- check that it does not say this cell line is not a duplicate from another cell line
- check that if it says this cell line is derived/children/father/samepatient from other cell lines, and that if we have any of the other cell lines, that the patient id is changed to be the same one for all (be sure that you are updating everywhere these patient ids are used)

In [176]:
len(ccle_refsamples)

6610

In [259]:
# If I have a previous samples I can update unknown data directly
# TODO: to functionalize
index=[]
notfound=[]
toupdate = {"sex":[],
"primary_disease":[],
"sm_id":[],
"cellosaurus_id":[],
"age":[],
"primary_site":[],
"subtype":[],
"subsubtype":[],
"origin":[],
"parent_cell_line":[''],
"matched_normal":[''],
"comments":[],
"participant_id":[]}
for k, val in samples.iterrows():
    dat = ccle_refsamples[ccle_refsamples['arxspan_id']==val['arxspan_id']]
    if len(dat)>0:
        index.append(k)
        for k, v in toupdate.items():
            toupdate[k].append(dat[k].tolist()[0])
    else:
        notfound.append(k)
# doing so..
for k, v in toupdate.items():
    samples.loc[index,k] =v
len(samples.loc[notfound].patient_id), samples.loc[notfound].patient_id.tolist()

(2, ['PT-6v9LD4Za', 'PT-fPnNIehK'])

In [187]:
# found same patient
a = ["ACH-000635","ACH-000717", "ACH-000864", "ACH-001042", "ACH-001547"]
b = ["ACH-002291","ACH-001672"]

In [260]:
# duplicate ach-id
dup = {"ACH-001620": "ACH-001605",
"ACH-001621": "ACH-001606"}

In [261]:
if any([i in samples.arxspan_id.tolist() for i in dup.keys()]):
    samples = changeCellLineNameInNewSet(new = samples, ref=ccle_refsamples, datatype="wes", dupdict=dup)

In [262]:
# If I have a previous samples I can update unknown data directly
index=[]
notfound=[]
toupdate = {"sex":[],
"primary_disease":[],
"sm_id":[],
"cellosaurus_id":[],
"age":[],
"primary_site":[],
"subtype":[],
"subsubtype":[],
"parent_cell_line":[''],
"matched_normal":[''],
"origin":[],
"comments":[],
"participant_id":[]}
for k, val in samples.loc[notfound].iterrows():
    dat = ccle_refsamples[ccle_refsamples['arxspan_id']==val['arxspan_id']]
    if len(dat)>0:
        index.append(k)
        for k, v in toupdate.items():
            toupdate[k].append(dat[k].tolist()[0])
    else:
        notfound.append(k)
# doing so..
for k, v in toupdate.items():
    samples.loc[index,k] =v
len(samples.loc[notfound].patient_id), samples.loc[notfound].patient_id.tolist()

(0, [])

In [270]:
samples.loc[notfound]

,internal_bam_filepath,internal_bai_filepath,stripped_cell_line_name,arxspan_id,sequencing_date,patient_id,crc32c_hash,size,version,datatype,sex,primary_disease,sm_id,cellosaurus_id,age,primary_site,subtype,subsubtype,origin,comments
sample_id,,,,,,,,,,,,,,,,,,,,
CDS-Qan1vz,gs://fc-9d2e10ea-1be3-4a23-a772-57854dbd1659/D...,gs://fc-9d2e10ea-1be3-4a23-a772-57854dbd1659/D...,WAOSEL,ACH-001707,737497,PT-fPnNIehK,y7Ygww==,20355332706,1,wes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [278]:
toupdate = {"sex":["Male"],
"primary_disease":["Leukemia"],
"cellosaurus_id":["CVCL_Y549"],
"age":['Adult'],
"primary_site":["haematopoietic_and_lymphoid_tissue"],
"subtype":["CLL"],
"subsubtype":["b_cell"],
"comments":["B-type chronic lymphocytic leukemia (CLL, Rai stage I at diagnosis)"],
"stripped_cell_line_name":["21MT2"],
"parent_cell_line":[''],
"matched_normal":[''],
"participant_id":['PT-y3RbI7uD']}

In [279]:
a  = pd.DataFrame(toupdate)
a['name'] = samples.loc[notfound,"stripped_cell_line_name"].tolist()
a

,sex,primary_disease,cellosaurus_id,age,primary_site,subtype,subsubtype,comments,stripped_cell_line_name,patient_id,name
0,Male,Leukemia,CVCL_Y549,Adult,haematopoietic_and_lymphoid_tissue,CLL,b_cell,"B-type chronic lymphocytic leukemia (CLL, Rai ...",21MT2,PT-y3RbI7uD,21MT2


In [280]:
# updating..
for k, v in toupdate.items():
    samples.loc[notfound,k] =v

In [288]:
# uploading to our bucket (now a new function)
h.changeToBucket(samples,'gs://cclebams/wes/', name_col= "index" , values=['internal_bam_filepath','internal_bai_filepath'], filetypes=['bam', 'bai'], catchdup=True, test=False)

/home/jeremie/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
sampes['baits'] = 'ice'

## Check that we have all the cell lines we expect for this release

This involves comparing to the list in the Google sheet "Cell Line Profiling Status."

_As the list cannot be parsed, we are not comparing it for now_

In [362]:
names=[]
subccle_refsamples = ccle_refsamples[ccle_refsamples['datatype'] == "wes"]
for k, val in samples.iterrows():
    val = val["arxspan_id"]
    names.append(val)
    samples.loc[k, 'version'] = len(subccle_refsamples[subccle_refsamples['arxspan_id'] == val]) + names.count(val)
samples['version'] = samples['version'].astype(int)

In [ ]:
ccle_refsamples = pd.read_csv('temp/updated_ref_samples.csv', index_col=0)

In [365]:
subccle_refsamples.sequencing_date = h.datetoint(subccle_refsamples.sequencing_date.values, split='/', order = "asc")
for k, val in samples.iterrows():
    loc = subccle_refsamples[subccle_refsamples.arxspan_id==val.arxspan_id]
    if len(loc)>0:
        if val.sequencing_date > 0:
            for i, v in loc.iterrows():
                if v.sequencing_date > val.sequencing_date:
                    ccle_refsamples.loc[i,'version']+=1
                    samples.loc[k, 'version']-=1
        else:
            if max(loc['size']) > val['size']:
                samples.loc[k, 'version'] = 1
                ccle_refsamples.loc[loc.index,'version'] = ccle_refsamples.loc[loc.index,'version'].values+1     

In [381]:
ccle_refsamples = ccle_refsamples.append(samples, sort=False)

In [ ]:
for val in samepatient:
    sub = ccle_refsamples[ccle_refsamples.arxspan_id.isin(val)]
    if len(set(sub.participant_id))>2:
        print('we found a missig participant relationship')
        # ccle_refsamples.loc[ccle_refsamples.index, "participant_id"]=sub.participant_id[0]

In [385]:
ccle_refsamples.to_csv('temp/updated_ref_samples.csv')

In [370]:
pairs = setupPairsFromSamples(samples, subccle_refsamples, extract={'patient_id':'participant_id'})

found 0 matched normals


In [372]:
#uploading new samples to mut
refwm = refwm.disable_hound()
refwm.upload_samples(samples)
refwm.upload_entities('pairs', pairs)
refwm.update_pair_set(pair_set_id=samplesetname,pair_ids=pairs.index)
sam = refwm.get_samples()

pair = refwm.get_pairs()
refwm.update_pair_set(pair_set_id='all',pair_ids=pair.index)
refwm.update_pair_set(pair_set_id='all_agilent',pair_ids=pair[pair["case_sample"].isin(sam[sam['baits']=="AGILENT"].index.tolist())].index)
refwm.update_pair_set(pair_set_id='all_ice',pair_ids=pair[pair["case_sample"].isin([i for i in sam[(sam['baits'] == "ICE") |(sam['baits'].isna())].index.tolist() if i != 'nan'])].index)
#creating a sample set
refwm.update_sample_set(sample_set_id=samplesetname, sample_ids=samples.index)
refwm.update_sample_set(sample_set_id='all', sample_ids=[i for i in sam.index.tolist() if i!='nan'])
refwm.update_sample_set(sample_set_id='all_agilent', sample_ids = sam[sam['baits'] == "AGILENT"].index.tolist())
refwm.update_sample_set(sample_set_id='all_ice', sample_ids=[i for i in sam[(sam['baits'] == "ICE") |(sam['baits'].isna())].index.tolist() if i != 'nan'])

Successfully imported 385 participants.
Successfully imported 414 samples.
Successfully imported 414 pairss.
Pair set "20Q4" (414 pairs) successfully updated.
Pair set "all" (2333 pairs) successfully updated.
Pair set "all_agilent" (1403 pairs) successfully updated.
Pair set "all_ice" (930 pairs) successfully updated.
Sample set "20Q4" (414 samples) successfully updated.
Sample set "all" (2333 samples) successfully updated.
Sample set "all_agilent" (1403 samples) successfully updated.
Sample set "all_ice" (930 samples) successfully updated.


In [374]:
#and CN
cnwm = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_WES_CN_hg38')
cnwm = cnwm.disable_hound()
cnwm.upload_samples(samples)
cnwm.upload_entities('pairs', pairs)
cnwm.update_pair_set(pair_set_id=samplesetname,pair_ids=pairs.index)
sam = cnwm.get_samples()

pair = cnwm.get_pairs()
cnwm.update_pair_set(pair_set_id='all',pair_ids=pair.index)
cnwm.update_pair_set(pair_set_id='all_agilent',pair_ids=pair[pair["case_sample"].isin(sam[sam['baits']=="AGILENT"].index.tolist())].index)
cnwm.update_pair_set(pair_set_id='all_ice',pair_ids=pair[pair["case_sample"].isin([i for i in sam[(sam['baits'] == "ICE") |(sam['baits'].isna())].index.tolist() if i != 'nan'])].index)
#creating a sample set
cnwm.update_sample_set(sample_set_id=samplesetname, sample_ids=samples.index)
cnwm.update_sample_set(sample_set_id='all', sample_ids=[i for i in sam.index.tolist() if i!='nan'])
cnwm.update_sample_set(sample_set_id='all_agilent', sample_ids = sam[sam['baits'] == "AGILENT"].index.tolist())
cnwm.update_sample_set(sample_set_id='all_ice', sample_ids=[i for i in sam[(sam['baits'] == "ICE") |(sam['baits'].isna())].index.tolist() if i != 'nan'])

Successfully imported 385 participants.
Successfully imported 414 samples.
Successfully imported 414 pairss.
Successfully imported 1 pair sets:
  * 20Q4 (414 pairs)
Pair set "all" (2333 pairs) successfully updated.
Pair set "all_agilent" (1403 pairs) successfully updated.
Pair set "all_ice" (930 pairs) successfully updated.
Successfully imported 1 sample sets:
  * 20Q4 (414 samples)
Sample set "all" (2333 samples) successfully updated.
Sample set "all_agilent" (1403 samples) successfully updated.
Sample set "all_ice" (930 samples) successfully updated.


# run the pipeline

We are using Dalmatian to send request to Terra, we are running a set of 5 functions To generate the mutation dataset:

*   For new samples in DepMap, run the ICE version of this task. CCLE2 samples used Agilent targets, so this pipeline should be used instead. The pipelines are identical in terms of their outputs, but the proper targets, baits, and pseudo normal should be used based on how the samples were sequenced.

    **ICE_CGA_Production_Analysis_Pipeline_Cell_Lines_copy** (cclf/CGA_Production_Analysis_Pipeline_Cell_Lines_debuggingSnapshot ID: 22) OR


    **AGILENT_CGA_Production_Analysis_Pipeline_Cell_Lines** (cclf/CGA_Production_Anablysis_Pipeline_Cell_Lines_debuggingSnapshot ID: 22)

*   **common_variant_filter** (breardon/common_variant_filterSnapshot ID: 3)
*   **filterMAF_on_CGA_pipeline** (gkugener/filterMAF_on_CGA_pipelineSnapshot ID: 8)
*   **aggregateMAFs_selectFields** (ccle_mg/aggregateMAFs_selectFieldsSnapshot ID: 1)

This outputs to be downloaded will be saved in the sample set that was run. The output we use for the release is:


*   **passedCGA_filteredMAF_aggregated** 

There are several other tasks in this workspace. In brief:



*   **CGA_Production_Analysis_Pipeline_Cell_Lines** (lelagina/CGA_Production_Analysis_Pipeline_Cell_LinesSnapshot ID: 12). This task is the same as the ICE and AGILENT prefixed version above, except that it relied on pulling the baits and targets to use from the metadata stored for the samples. Having AGILENT and ICE versions specified made the uploading and running process easier.
*   **SANGER_CGA_Production_Analysis_Pipeline_Cell_Lines** (cclf/CGA_Production_Analysis_Pipeline_Cell_Lines_debuggingSnapshot ID: 22). This task was trying to run the CGA pipeline on the Sanger WES data, using a Sanger pseudo normal. In its current implementation, this task fails to complete for the samples.
*   **UNFILTERED_aggregateMAFs_selectFields** (ccle_mg/aggregateMAFs_selectFieldsSnapshot ID: 1). Aggregates the MAF outputted by the CGA cell line pipeline prior to the common variant filter and germline filtering tasks. This can give us insight to which mutations are getting filtered out when. We may want to potentially include this MAF in the release so people can see why certain mutations of interest may be getting filtered out.
*   WES_DM_Mutation_Calling_Pipeline_(standard |expensive) (gkugener/WES_DM_Mutation_Calling_PipelineSnapshot ID: 2). This was a previous mutation calling pipeline implemented for CCLE. We do not use this pipeline any more as the CGA pipeline looks better.
*   aggregate_filterMAF_CGA (CCLE/aggregate_filterMAF_CGASnapshot ID: 1). An aggregation MAF task that we used in the past. We do not use this task anymore.
*   calculate_mutational_burden (breardon/calculate_mutational_burdenSnapshot ID: 21). This task can be used to calculate the mutational rate of the samples. We do not make use of this data in the release although it could be of interest.
*   summarizeWigFile (breardon/summarizeWigFileSnapshot ID: 5). CCLF ran this task (might be necessary for the mutational burden task). For our workflow, we do not run it.

## On Terra

In [390]:
samplesetname

'20Q4'

In [399]:
submission_id1 = refwm.create_submission("CGA_WES_CCLE_ICE", samplesetname, 'sample_set', expression='this.samples')

Successfully created submission c4de93d4-c807-473b-966c-5f7436ce8ab2.


### Germline

In [397]:
submission_id2 = refwm.create_submission("cnn-variant-filter", samplesetname, 'sample_set', expression='this.samples')

Successfully created submission 97f48fee-8011-4112-b3ca-9dd5f036ba52.


### copy pairs data to sample data

In [ ]:
pairs = refwm.get_pairs()

In [ ]:
pairs = pairs[pairs.index.isin(tokeep)]
pairs = pairs[~pairs['mutation_validator_validated_maf'].isna()]
pairs = pairs.drop(columns=['case_sample','control_sample','participant_id'])
pairs.index = [i.split('_')[0] for i in pairs.index]

In [ ]:
refwm.update_sample_attributes(pairs)

continuing

In [413]:
terra.waitForSubmission(refworkspace, submission_id1)
submission_id1 = refwm.create_submission("common_variant_filter", samplesetname, 'sample_set', expression='this.samples')

Successfully created submission 2b7316ae-8d11-4cb9-9143-efd9c7354cb5.


In [404]:
terra.waitForSubmission(refworkspace, submission_id2)
submission_id2 = refwm.create_submission("aggregate_vcfs", "all")

Successfully created submission a7c71e70-dc82-4b95-92c5-3d98b7438837.


In [414]:
terra.waitForSubmission(refworkspace, submission_id1)
submission_id1 = refwm.create_submission("filterMAF_on_CGA_pipeline", samplesetname,'sample_set',expression='this.samples')

KeyboardInterrupt: 

### filtered

In [411]:
terra.waitForSubmission(refworkspace, submission_id1)
submission_id1 = refwm.create_submission("aggregateMAFs_selectFields_filtered", "all")

1.0 of jobs Succeeded in submission 0.
Successfully created submission 8f3249b7-99a7-40f9-a4ed-010fbed48389.


### unfiltered

In [412]:
submission_id3 = refwm.create_submission("aggregateMAFs_selectFields_unfiltered", "all")

Successfully created submission 381337f5-dbd0-4d0d-8d4b-eda938904ce7.


In [ ]:
terra.waitForSubmission(refworkspace, [submission_id1,submission_id2, submission_id3])

### Save the workflow configurations used

In [8]:
terra.saveConfigs(refworkspace,'./data/'+samplesetname+'/Mutconfig')

## On local


### Remove some datafile to save money¶

In [ ]:
res = refwm.get_samples()
toremove = ["fixedmate_bam"]
for val in toremove:
    refwm.disable_hound().delete_entity_attributes('sample', res[val], delete_files=True)

In [ ]:
! gsutil -m rm "gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/**/call-tumorMM_Task/*.cleaned.bam"

In [ ]:
# sometimes it does not work; so better check again
a = res.fixedmate_bam
a = [i for i in a if i is not np.nan]
gcp.rmFiles(a)

### downloading from terra

In [ ]:
sam = refwm.get_samples()

In [ ]:
nowes = set(mutations.DepMap_ID)-set(sam.arxspan_id)
nowes

In [ ]:
mutations.columns

In [ ]:
nothing = nows -set(ccle_refsamples.arxspan_id)
nothing

In [ ]:
set(mutations[mutations.DepMap_ID.isin(nothing) & ~mutations.SangerWES_AC.isna()].DepMap_ID)

### get QC files

In [13]:
dataMut = getQC(workspace=refworkspace ,only=[], qcname=["gatk_cnv_all_plots", "lego_plotter_pngs", "copy_number_qc_report", "ffpe_OBF_figures", "mut_legos_html", "oxoG_OBF_figures", "tumor_bam_base_distribution_by_cycle_metrics", "tumor_bam_converted_oxog_metrics"])

In [14]:
dataBam = getQC(workspace=refworkspace ,only=[], qcname=[ "tumor_bam_alignment_summary_metrics", "tumor_bam_bait_bias_summary_metrics", "tumor_bam_gc_bias_summary_metrics", "tumor_bam_hybrid_selection_metrics", "tumor_bam_insert_size_histogram", "tumor_bam_insert_size_metrics", "tumor_bam_pre_adapter_summary_metrics", "tumor_bam_quality_by_cycle_metrics", "tumor_bam_quality_distribution_metrics", "tumor_bam_quality_yield_metrics"])

In [6]:
wrongsamples = ['CDS-jqOvtj', 'CDS-FRxdcH', 'CDS-6Yy3Yj', 'CDS-CuJ0f8', 'CDS-rLRUbG', 'CDS-PdUZxY', 'CDS-eUqT7L', 'CDS-KbbgMb', 'CDS-6da3hu', 'CDS-fXMRF9', 'CDS-CMenCH', 'CDS-MLJbT2', 'CDS-QVhVDT', 'CDS-XevQNc', 'CDS-0pZb0j', 'CDS-6l3V79', 'CDS-MnF3x8', 'CDS-ihI7Dp', 'CDS-34hKv3', 'CDS-TyWjJs', 'CDS-4sr6RL', 'CDS-M8xDMS', 'CDS-TpDBjm', 'CDS-W80jkV', 'CDS-agZcmk', 'CDS-cYWYp7', 'CDS-IJnjkY', 'CDS-0aJ4Yh', 'CDS-txTRwz', 'CDS-gIMBax', 'CDS-1p2nnc', 'CDS-KQDgIV', 'CDS-Eq9UNX', 'CDS-3M6Pq9', 'CDS-qZsCuJ', 'CDS-0lfqVz', 'CDS-o4dXGr', 'CDS-uQ8nnX', 'CDS-iqPqOr', 'CDS-Dkl8OF', 'CDS-Hj3xAa', 'CDS-3WygAj', 'CDS-oHu1Ik', 'CDS-X3c4UY', 'CDS-PYw8ID', 'CDS-Sp18uD', 'CDS-leGxSD', 'CDS-SJq3p4', 'CDS-no7ysz', 'CDS-UnDaBI', 'CDS-eowEZF', 'CDS-HNytLD', 'CDS-KYkMDa', 'CDS-OgPf0h', 'CDS-OCkOqy', 'CDS-QU7ftt', 'CDS-iEULQm', 'CDS-ODmXrP', 'CDS-YMIv9D', 'CDS-5rD8XC', 'CDS-QXBhht', 'CDS-9XPgHB', 'CDS-Ig6N9S', 'CDS-UtrDTK', 'CDS-nby0QM', 'CDS-49azaP', 'CDS-9qDPiX', 'CDS-KgRznV', 'CDS-picEuX', 'CDS-L0pDPl', 'CDS-kxNZ5S', 'CDS-1djAlo', 'CDS-YYLKZ0', 'CDS-pXMN9C', 'CDS-gRA4SM', 'CDS-QHp4h4', 'CDS-B0qAaq', 'CDS-1b1Hxk', 'CDS-5wYxZS', 'CDS-cyuMYb', 'CDS-XQkXf4', 'CDS-7PFldq',
'CDS-3EBt51', 'CDS-UV1pVE', 'CDS-WedVJA', 'CDS-WfjTcJ', 'CDS-bntBUl', 'CDS-cAEii6', 'CDS-d18Xie', 'CDS-dpub1O', 'CDS-yPSmxb']

In [16]:
for k,v in dataMut.items():
    if k =='nan':
        continue
    curr = ccle_refsamples.loc[k,'processing_qc']
    curr = set(curr[1:-1].replace("'","").split(', '))
    curr = set(v) | curr
    ccle_refsamples.loc[k,'processing_qc'] = str(list(curr))
for k,v in dataBam.items():
    if k =='nan':
        continue
    curr = ccle_refsamples.loc[k,'processing_qc']
    curr = set(curr[1:-1].replace("'","").split(', '))
    curr = set(v) | curr
    ccle_refsamples.loc[k,'bam_qc'] = str(list(curr))
#ccle_refsamples.to_csv('temp/newrefWES.csv')

In [7]:
res = refwm.get_sample_sets().loc["all"]
res

/home/jeremie/miniconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


filtered_CGA_MAF_aggregated      gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b7...
samples                          [CDS-00rz9N, CDS-01bI6z, CDS-02waxZ, CDS-04TUV...
merged_mutations                 gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b7...
merged_vcf                       gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b7...
unfiltered_CGA_MAF_aggregated    gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b7...
Name: all, dtype: object

### retrieving RNAseq vcfs

In [ ]:
rnamutations = dm.WorkspaceManager(rnaworkspace).get_sample_sets().loc['All_samples']['merged_vcf']
! gsutil cp $rnamutations "temp/rna_mutation_unfiltered_terra_merged.vcf"

### retrieving germline mutations

In [8]:
snps = res['merged_vcf']
! gsutil cp $snps gs://cclebams/germline_data/wes.all.called.vcf
! gsutil -m acl ch -ru taiga-892@cds-logging.iam.gserviceaccount.com:R gs://cclebams/germline_data/wes.all.called.vcf

Copying gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/a7c71e70-dc82-4b95-92c5-3d98b7438837/aggregate_vcfs/434a1e9d-487e-41a7-a3ee-eaaea7233573/call-aggregate/all.called.vcf [Content-Type=text/vcard]...
/ [1 files][ 49.6 GiB/ 49.6 GiB]                                                
Operation completed over 1 objects/49.6 GiB.                                     
Updated ACL on gs://cclebams/germline_data/wes.all.called.vcf


### rertrievinng filtered mutations

In [10]:
filtered = res['filtered_CGA_MAF_aggregated']
! gsutil cp $filtered "temp/mutation_filtered_terra_merged.txt"

Copying gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/8f3249b7-99a7-40f9-a4ed-010fbed48389/aggregateMAFs_selectFields_workflow/756e9aae-f842-4cee-b1d6-2cc03198842d/call-aggregateMAFs_selectFields/all.mergedMAF.txt...
- [1 files][308.8 MiB/308.8 MiB]                                                
Operation completed over 1 objects/308.8 MiB.                                    


In [8]:
mutations = mutations[~mutations.Tumor_Sample_Barcode.isin(wrongsamples)]

In [9]:
mutations = pd.read_csv('temp/mutation_filtered_terra_merged.txt',sep='\t') 
print(mutations.columns[:10])
renaming = removeOlderVersions(names = set(mutations['Tumor_Sample_Barcode']), refsamples = refwm.get_samples(), arxspan_id = "arxspan_id", version="version")
print(len(mutations[mutations['Chromosome']=='0']))

Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'Center', 'NCBI_Build', 'Chromosome',
       'Start_position', 'End_position', 'Strand', 'Variant_Classification',
       'Variant_Type'],
      dtype='object')
removed 535 duplicate samples
0


# postprocessing


Here, rather than rerunning the entire analysis, because we know we are adding only WES samples, we can download the previous release's MAF, add the samples, update any annotations, and perform any global filters at the end.

First we need to do an additional step of filtering on coverage and number 

- readMutations
- createSNPs
- addToMainMutation
- filterAllelicFraction
- filterMinCoverage
- mergeAnnotations
- addAnnotation
- maf_add_variant_annotations
- mutation_maf_to_binary_matrix (x3)

In [10]:
mutations = mutations.rename(columns={"i_ExAC_AF":"ExAC_AF","Tumor_Sample_Barcode":'DepMap_ID',"Tumor_Seq_Allele2":"Tumor_Allele"}).drop(columns=['Center','Tumor_Seq_Allele1'])

In [11]:
mutations = annotate_likely_immortalized(mutations, TCGAlocs = ['TCGAhsCnt',
'COSMIChsCnt'], max_recurrence=0.05 ,min_tcga_true_cancer=5)

In [12]:
mutations['CGA_WES_AC'] = [str(i[0]) + ':' + str(i[1]) for i in np.nan_to_num(mutations[['t_alt_count','t_ref_count']].values,0).astype(int)]

In [13]:
mutations = filterCoverage(mutations, loc=['CGA_WES_AC'], sep=':',cov=2)

In [14]:
mutations = filterAllelicFraction(mutations, loc=['CGA_WES_AC'], sep=':',frac=0.1)

In [15]:
#Count the total number of mutations per cell line, split by type (SNP, INS, DEL)
#Count the total number of mutations observed by position

In [16]:
mutations = addAnnotation(mutations, NCBI_Build='37', Strand="+")

In [17]:
mutations.to_csv('temp/wes_somatic_mutations_withduplicates_'+samplesetname+'.csv', index=False)

In [18]:
mutations = mutations[mutations.DepMap_ID.isin(renaming.keys())].replace(renaming)

### Adding WGS's exonic mutation

In [27]:
wgsemutations = pd.read_csv('temp/wgs_somatic_mutations_'+samplesetname+'.csv')

In [30]:
#for now we keep WES if we have them
toadd = set(wgsemutations.DepMap_ID) - set(mutations.DepMap_ID)
toadd

{'ACH-000195',
 'ACH-001046',
 'ACH-001078',
 'ACH-001553',
 'ACH-001705',
 'ACH-001828',
 'ACH-002013',
 'ACH-002014',
 'ACH-002512'}

In [34]:
mutations = mutations.append(wgsemutations[wgsemutations.DepMap_ID.isin(toadd)])

In [ ]:
mutations.to_csv('temp/wes_somatic_mutations_all_'+samplesetname+'.csv', index=False)

In [6]:
mutations = pd.read_csv('temp/wes_somatic_mutations_all_'+samplesetname+'.csv')

In [10]:
mutations

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,tumor_f,isDeleterious,isTCGAhotspot,TCGAhsCnt,isCOSMIChotspot,COSMIChsCnt,ExAC_AF,PASS,is_likely_immortalization,CGA_WES_AC
0,PRDM2,7799,37,1,14108651,14108651,+,Missense_Mutation,SNP,C,...,0.492647,False,False,NaN,False,0.0,NaN,True,False,335:345
1,TMEM51,55092,37,1,15541888,15541888,+,Missense_Mutation,SNP,A,...,0.192308,False,False,NaN,False,0.0,NaN,True,False,5:21
2,EPHA2,1969,37,1,16464806,16464806,+,Missense_Mutation,SNP,G,...,0.304348,False,False,NaN,False,0.0,0.000016,True,False,7:16
3,SPATA21,374955,37,1,16727249,16727249,+,Silent,SNP,G,...,0.104396,False,False,NaN,False,0.0,NaN,True,False,19:163
4,EPHB2,2048,37,1,23107922,23107922,+,Missense_Mutation,SNP,A,...,0.409091,False,False,NaN,False,0.0,NaN,True,False,18:26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019205,CD7,924,37,17,80274142,80274168,+,In_Frame_Del,DEL,CAGGGAGGGCAGAGGCTGCTGGCGGGT,...,0.476190,False,False,NaN,False,0.0,NaN,True,True,10:11
1019206,YIPF2,78992,37,19,11036411,11036411,+,Frame_Shift_Del,DEL,A,...,0.620690,True,False,NaN,False,0.0,NaN,True,False,18:11
1019207,GYS1,2997,37,19,49494632,49494636,+,Frame_Shift_Del,DEL,TGGGT,...,0.545455,True,False,NaN,False,0.0,NaN,True,False,18:15
1019208,ISOC2,79763,37,19,55972878,55972878,+,Splice_Site,DEL,C,...,0.411765,True,False,NaN,False,0.0,NaN,True,False,14:20


In [11]:
mafToMat(mutations[(mutations.isDeleterious) & (mutations['tumor_f']>0.25)]).T.to_csv('temp/wes_somatic_mutations_deleterious_matrix.csv')
mafToMat(mutations[~(mutations.isDeleterious | mutations.isCOSMIChotspot | mutations.isTCGAhotspot | mutations['Variant_Classification']=='Silent') & (mutations['tumor_f']>0.25)]).T.to_csv('temp/wes_somatic_mutations_other_matrix.csv')
mafToMat(mutations[(mutations.isCOSMIChotspot | mutations.isTCGAhotspot) &(mutations['tumor_f']>0.25)]).T.to_csv('temp/wes_somatic_mutations_hotspot_matrix.csv')

In [12]:
mafToMat(mutations[(mutations.isDeleterious) & (mutations['tumor_f']>0.25)], boolify=True).astype(int).T.to_csv('temp/wes_somatic_mutations_deleterious_boolmatrix.csv')
mafToMat(mutations[~(mutations.isDeleterious | mutations.isCOSMIChotspot | mutations.isTCGAhotspot | mutations['Variant_Classification']=='Silent') & (mutations['tumor_f']>0.25)], boolify=True).astype(int).T.to_csv('temp/wes_somatic_mutations_other_boolmatrix.csv')
mafToMat(mutations[(mutations.isCOSMIChotspot | mutations.isTCGAhotspot) & (mutations['tumor_f']>0.25)], boolify=True).astype(int).T.to_csv('temp/wes_somatic_mutations_hotspot_boolmatrix.csv')

In [7]:
legacy_hybridcapture = tc.get(name='mutations-da6a', file='legacy_hybridcapture_somatic_mutations').drop(columns=['Unnamed: 0',"Tumor_Sample_Barcode"]).rename(columns={'Tumor_Seq_Allele1':'Tumor_Allele'})
legacy_raindance = tc.get(name='mutations-da6a', file='legacy_raindance_somatic_mutations').drop(columns=['Unnamed: 0',"Tumor_Sample_Barcode"]).rename(columns={'Tumor_Seq_Allele1':'Tumor_Allele'})
legacy_rna = tc.get(name='mutations-da6a', file='legacy_rna_somatic_mutations').drop(columns=['Unnamed: 0',"Tumor_Sample_Barcode"]).rename(columns={'Tumor_Seq_Allele1':'Tumor_Allele'})
legacy_wes_sanger = tc.get(name='mutations-da6a', file='legacy_wes_sanger_somatic_mutations').drop(columns=['Unnamed: 0',"Tumor_Sample_Barcode"]).rename(columns={'Tumor_Seq_Allele1':'Tumor_Allele'})
legacy_wgs_exoniconly = tc.get(name='mutations-da6a', file='legacy_wgs_exoniconly_somatic_mutations').drop(columns=['Unnamed: 0',"Tumor_Sample_Barcode"]).rename(columns={'Tumor_Seq_Allele1':'Tumor_Allele'})

solving issues with the legacy datasets

In [9]:
legacy_hybridcapture[legacy_hybridcapture.DepMap_ID=='ACH-001011']

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,Protein_Change,isDeleterious,isTCGAhotspot,TCGAhsCnt,isCOSMIChotspot,COSMIChsCnt,ExAC_AF,HC_AC,Variant_annotation,DepMap_ID
3863,ALS2,57679,37,2,202575734,202575734,+,Missense_Mutation,SNP,T,...,p.I1368F,False,False,0.0,False,0.0,NaN,39:33,other non-conserving,ACH-001011
4553,AR,367,37,X,66765525,66765525,+,Silent,SNP,T,...,p.A179A,False,False,0.0,False,0.0,NaN,21:15,silent,ACH-001011
5536,ASTN1,460,37,1,176853552,176853552,+,Missense_Mutation,SNP,A,...,p.I1058N,False,False,0.0,False,0.0,NaN,41:40,other non-conserving,ACH-001011
6106,ATP13A5,344905,37,3,192992854,192992854,+,Missense_Mutation,SNP,G,...,p.Q1212E,False,False,0.0,False,0.0,NaN,104:120,other non-conserving,ACH-001011
10077,CARD14,79092,37,17,78182096,78182096,+,Silent,SNP,C,...,p.I989I,False,False,0.0,False,0.0,0.000008,40:0,silent,ACH-001011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80904,TTN,7273,37,2,179549633,179549633,+,Splice_Site,SNP,C,...,p.V10535L,True,False,0.0,False,0.0,NaN,10:17,damaging,ACH-001011
82767,UNC13C,440279,37,15,54685278,54685278,+,Missense_Mutation,SNP,G,...,p.Q1582H,False,False,0.0,False,0.0,NaN,18:10,other non-conserving,ACH-001011
82953,UNC79,57578,37,14,94088426,94088426,+,Missense_Mutation,SNP,C,...,p.S1616L,False,True,3.0,False,0.0,0.000008,73:77,other non-conserving,ACH-001011
83671,VRK1,7443,37,14,97321627,97321627,+,Missense_Mutation,SNP,G,...,p.E215K,False,False,0.0,False,0.0,NaN,40:77,other non-conserving,ACH-001011


In [39]:
legacy_hybridcapture.loc[legacy_hybridcapture[legacy_hybridcapture['Variant_Classification'].isna()].index,'Variant_Classification']='Missense_Mutation'

In [11]:
legacy_wes_sanger[legacy_wes_sanger.DepMap_ID=="ACH-002396"]

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,Protein_Change,isDeleterious,isTCGAhotspot,TCGAhsCnt,isCOSMIChotspot,COSMIChsCnt,ExAC_AF,SangerWES_AC,Variant_annotation,DepMap_ID
121,A2M,2,37,12,9246084,9246084,+,Nonsense_Mutation,SNP,C,...,p.W739*,True,False,0.0,False,0.0,NaN,16:12,damaging,ACH-002396
251,A2ML1,144568,37,12,9010695,9010695,+,Missense_Mutation,SNP,G,...,p.M1087I,False,False,0.0,False,0.0,NaN,16:25,other non-conserving,ACH-002396
905,AATF,26574,37,17,35413952,35413952,+,Silent,SNP,C,...,p.G557G,False,False,0.0,False,0.0,NaN,21:43,silent,ACH-002396
1022,ABCA1,19,37,9,107558438,107558438,+,Missense_Mutation,SNP,A,...,p.F1760L,False,False,0.0,False,0.0,NaN,5:12,other non-conserving,ACH-002396
1162,ABCA10,10349,37,17,67178424,67178424,+,Missense_Mutation,SNP,C,...,p.R880K,False,False,0.0,False,0.0,NaN,9:19,other non-conserving,ACH-002396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633846,ZSCAN5C,649137,37,19,56717508,56717508,+,Missense_Mutation,SNP,G,...,p.D76N,False,False,0.0,False,0.0,0.000008,9:15,other non-conserving,ACH-002396
633957,ZSWIM3,140831,37,20,44505644,44505644,+,Silent,SNP,G,...,p.S149S,False,False,0.0,False,0.0,0.000008,82:26,silent,ACH-002396
634219,ZSWIM8,23053,37,10,75559943,75559943,+,Missense_Mutation,SNP,C,...,p.Q1609E,False,False,0.0,False,0.0,NaN,125:34,other non-conserving,ACH-002396
634528,ZYX,7791,37,7,143087031,143087031,+,Silent,SNP,C,...,p.A525A,False,False,0.0,False,0.0,NaN,42:49,silent,ACH-002396


In [87]:
legacy_wgs_exoniconly.loc[legacy_wgs_exoniconly[legacy_wgs_exoniconly['Genome_Change'].isna()].index, 'Genome_Change'] = ['g.chr'+str(i.Chromosome)+":"+str(i.Start_position)+i.Reference_Allele+">"+i.Tumor_Allele for _, i in legacy_wgs_exoniconly[legacy_wgs_exoniconly['Genome_Change'].isna()].iterrows()]

In [112]:
legacy_wes_sanger.loc[legacy_wes_sanger[legacy_wes_sanger['Genome_Change'].isna()].index, 'Genome_Change'] = ['g.chr'+str(i.Chromosome)+":"+str(i.Start_position)+i.Reference_Allele+">"+i.Tumor_Allele for _, i in legacy_wes_sanger[legacy_wes_sanger['Genome_Change'].isna()].iterrows()]

In [85]:
legacy_raindance.loc[legacy_raindance[legacy_raindance['Genome_Change'].isna()].index, 'Genome_Change'] = ['g.chr'+str(i.Chromosome)+":"+str(i.Start_position)+i.Reference_Allele+">"+i.Tumor_Allele for _, i in legacy_raindance[legacy_raindance['Genome_Change'].isna()].iterrows()]

In [83]:
legacy_hybridcapture.loc[legacy_hybridcapture[legacy_hybridcapture['Genome_Change'].isna()].index, 'Genome_Change'] = ['g.chr'+str(i.Chromosome)+":"+str(i.Start_position)+i.Reference_Allele+">"+i.Tumor_Allele for _, i in legacy_hybridcapture[legacy_hybridcapture['Genome_Change'].isna()].iterrows()]

In [ ]:
legacy_rna.loc[legacy_rna[legacy_rna['Genome_Change'].isna()].index, 'Genome_Change'] = ['g.chr'+str(i.Chromosome)+":"+str(i.Start_position)+i.Reference_Allele+">"+i.Tumor_Allele for _, i in legacy_rna[legacy_rna['Genome_Change'].isna()].iterrows()]

In [ ]:
todrop = []
for val in h.dups(legacy_rna.loci):
    todrop.append(legacy_rna[legacy_rna.loci==val].index[0])
legacy_rna = legacy_rna.drop(todrop)

In [116]:
merged = mergeAnnotations(mutations, legacy_hybridcapture, useSecondForConflict=True, dry_run=False)
merged = mergeAnnotations(merged, legacy_raindance, useSecondForConflict=True, dry_run=False)
merged = mergeAnnotations(merged, legacy_rna, useSecondForConflict=False, dry_run=False)
merged = mergeAnnotations(merged, legacy_wes_sanger, useSecondForConflict=False, dry_run=False)

found 467 nonmatching mutations
found 3 nonmatching mutations
found 282 nonmatching mutations
found 255 nonmatching mutations


In [117]:
merged = mergeAnnotations(merged, legacy_wgs_exoniconly, useSecondForConflict=False, dry_run=False)

found 186 nonmatching mutations


In [118]:
mutation_groups={
"other conserving": ["5'Flank", "Intron", "IGR", "3'UTR", "5'UTR"],
"other non-conserving":["In_Frame_Del", "In_Frame_Ins", "Stop_Codon_Del", "Stop_Codon_Ins", "Missense_Mutation", "Nonstop_Mutation"],
'silent': ['Silent'],
"damaging":['De_novo_Start_OutOfFrame','Frame_Shift_Del','Frame_Shift_Ins', 'Splice_Site', 'Start_Codon_Del', 'Start_Codon_Ins', 'Start_Codon_SNP','Nonsense_Mutation']
}

In [119]:
rename = {}
for k,v in mutation_groups.items():
    for e in v:
        rename[e] = k
merged['Variant_annotation'] = [rename[i] for i in merged['Variant_Classification'].tolist()]

### Compare to previous release

I would run some checks here comparing the results to the previous releases MAF. Namely:

- Count the total number of mutations per cell line, split by type (SNP, INS, DEL)
- Count the total number of mutations observed by position (group by chromosome, start position, end position and count the number of mutations)
- Look at specific differences between the two MAFs (join on DepMap_ID, Chromosome, Start position, End position, Variant_Type). I would do this for WES only

In [311]:
prevmut = tc.get(name='depmap-mutation-calls-9be3', version=prevversion, file='depmap_'+prevname+'_mutation_calls')

Taiga404Exception: Datafile not found...check the file name?

In [ ]:
prevprev= tc.get(name='depmap-mutation-calls-9be3', file= "depmap_"+prevprevname+"_mutation_calls", version = prevprevversion)

In [127]:
set(merged.DepMap_ID) - set(merged[~(merged['CGA_WES_AC'].isna() & merged['SangerWES_AC'].isna() & merged['WGS_AC'].isna())].DepMap_ID)

{'ACH-000003',
 'ACH-000016',
 'ACH-000033',
 'ACH-000034',
 'ACH-000049',
 'ACH-000057',
 'ACH-000064',
 'ACH-000071',
 'ACH-000077',
 'ACH-000084',
 'ACH-000088',
 'ACH-000170',
 'ACH-000185',
 'ACH-000194',
 'ACH-000216',
 'ACH-000229',
 'ACH-000230',
 'ACH-000299',
 'ACH-000300',
 'ACH-000306',
 'ACH-000333',
 'ACH-000413',
 'ACH-000494',
 'ACH-000526',
 'ACH-000539',
 'ACH-000575',
 'ACH-000578',
 'ACH-000600',
 'ACH-000612',
 'ACH-000629',
 'ACH-000642',
 'ACH-000685',
 'ACH-000690',
 'ACH-000710',
 'ACH-000731',
 'ACH-000737',
 'ACH-000742',
 'ACH-000850',
 'ACH-000854',
 'ACH-000904',
 'ACH-000923',
 'ACH-000931',
 'ACH-001011',
 'ACH-001036',
 'ACH-001042',
 'ACH-001043',
 'ACH-001044',
 'ACH-001047',
 'ACH-001072',
 'ACH-001091',
 'ACH-001092',
 'ACH-001094',
 'ACH-001108',
 'ACH-001109',
 'ACH-001130',
 'ACH-001142',
 'ACH-001146',
 'ACH-001150',
 'ACH-001183',
 'ACH-001187',
 'ACH-001188',
 'ACH-001207',
 'ACH-001214',
 'ACH-001230',
 'ACH-001234'}

In [124]:
merged[merged.DepMap_ID=="ACH-000458"].sum(0)

Hugo_Symbol                  PLEKHN1ATAD3BMIB2SLC35E2BWRAP73CHD5RPL22RERESL...
Entrez_Gene_Id                                                       349125011
Chromosome                   1111111111111111111111111111111111111111111111...
Start_position                                                     50180743934
End_position                                                       50180744167
Strand                       ++++++++++++++++++++++++++++++++++++++++++++++...
Variant_Classification       SilentSilentSilentMissense_MutationSilentMisse...
Variant_Type                 SNPSNPSNPSNPSNPSNPSNPSNPSNPSNPSNPSNPSNPSNPSNPS...
Reference_Allele             GCGACCGGCGGCCGGGCCGGGCCCCT-CCCGTCCGCCGTAGG-TGC...
Tumor_Allele                 ATCGTTAATAATTAAATTCAATTTTGATTTTCTGAAGTAGATAAAT...
Genome_Change                g.chr1:908930G>Ag.chr1:1431080C>Tg.chr1:156395...
Annotation_Transcript        ENST00000379410.3ENST00000308647.7ENST00000505...
DepMap_ID                    ACH-000458ACH-000458ACH

In [128]:
merged[(merged["Hugo_Symbol"]=="ACOT4")&(merged['Start_position']==74058831)]

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,ExAC_AF,PASS,is_likely_immortalization,CGA_WES_AC,HC_AC,Variant_annotation,RD_AC,RNAseq_AC,SangerWES_AC,WGS_AC
726737,ACOT4,122970,37,14,74058831,74058831,+,Silent,SNP,C,...,NaN,True,False,161:0,NaN,silent,NaN,NaN,NaN,NaN
214446,ACOT4,122970,37,14,74058831,74058831,+,Silent,SNP,C,...,NaN,True,False,6:0,NaN,silent,NaN,NaN,NaN,NaN
219527,ACOT4,122970,37,14,74058831,74058831,+,Silent,SNP,C,...,NaN,True,False,4:10,NaN,silent,NaN,NaN,NaN,NaN
450951,ACOT4,122970,37,14,74058831,74058831,+,Silent,SNP,C,...,NaN,True,False,11:0,NaN,silent,NaN,NaN,NaN,NaN
107644,ACOT4,122970,37,14,74058831,74058831,+,Silent,SNP,C,...,NaN,True,False,4:2,NaN,silent,NaN,NaN,NaN,NaN
716089,ACOT4,122970,37,14,74058831,74058831,+,Silent,SNP,C,...,NaN,True,False,54:111,NaN,silent,NaN,NaN,NaN,NaN
599060,ACOT4,122970,37,14,74058831,74058831,+,Silent,SNP,C,...,NaN,True,False,116:116,NaN,silent,NaN,NaN,NaN,NaN
732270,ACOT4,122970,37,14,74058831,74058831,+,Silent,SNP,C,...,NaN,True,False,7:3,NaN,silent,NaN,NaN,NaN,NaN
1355588,ACOT4,122970,37,14,74058831,74058831,+,Silent,SNP,C,...,NaN,True,False,143:73,NaN,silent,NaN,NaN,NaN,NaN
492796,ACOT4,122970,37,14,74058831,74058831,+,Silent,SNP,C,...,NaN,True,False,78:133,NaN,silent,NaN,NaN,NaN,NaN


In [129]:
merged[merged.DepMap_ID=="ACH-001546"][merged.columns[-17:]]

,t_ref_count,tumor_f,isDeleterious,isTCGAhotspot,TCGAhsCnt,isCOSMIChotspot,COSMIChsCnt,ExAC_AF,PASS,is_likely_immortalization,CGA_WES_AC,HC_AC,Variant_annotation,RD_AC,RNAseq_AC,SangerWES_AC,WGS_AC


### Do some checks and manual rescuing

In [342]:
mutations[mutations.DepMap_ID=="ACH-003000"]

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,TCGAhsCnt,isCOSMIChotspot,COSMIChsCnt,ExAC_AF,PASS,is_likely_immortalization,CGA_WES_AC,ind,loci,Variant_annotation
1331264,SAMD11,148398,37,1,877951,877951,+,Silent,SNP,C,...,NaN,False,0.0,NaN,True,False,34:175,ACH-003000_g.chr1:877951C>T,ACH-003000_1_877951,silent
1331266,PAX7,5081,37,1,19027246,19027246,+,Missense_Mutation,SNP,G,...,NaN,False,0.0,0.000033,True,False,10:35,ACH-003000_g.chr1:19027246G>A,ACH-003000_1_19027246,other non-conserving
1331268,ZSCAN20,7579,37,1,33960120,33960120,+,Missense_Mutation,SNP,A,...,NaN,False,0.0,NaN,True,False,19:91,ACH-003000_g.chr1:33960120A>G,ACH-003000_1_33960120,other non-conserving
1331269,DAB1,1600,37,1,57611006,57611006,+,Missense_Mutation,SNP,G,...,NaN,False,0.0,NaN,True,False,144:50,ACH-003000_g.chr1:57611006G>A,ACH-003000_1_57611006,other non-conserving
1331270,FGGY,55277,37,1,59787280,59787280,+,Missense_Mutation,SNP,G,...,NaN,False,0.0,NaN,True,False,112:126,ACH-003000_g.chr1:59787280G>T,ACH-003000_1_59787280,other non-conserving
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1331681,ASPG,374569,37,14,104559916,104559917,+,Frame_Shift_Ins,INS,-,...,NaN,False,0.0,NaN,True,False,106:57,ACH-003000_g.chr14:104559916_104559917insTT,ACH-003000_14_104559916,damaging
1331682,DENND4A,10260,37,15,66048491,66048491,+,Frame_Shift_Del,DEL,G,...,NaN,False,0.0,NaN,True,False,19:50,ACH-003000_g.chr15:66048491delG,ACH-003000_15_66048491,damaging
1331683,NUDT7,283927,37,16,77775529,77775529,+,Frame_Shift_Del,DEL,G,...,NaN,False,0.0,NaN,True,False,41:124,ACH-003000_g.chr16:77775529delG,ACH-003000_16_77775529,damaging
1331684,TICAM1,148022,37,19,4817288,4817289,+,In_Frame_Ins,INS,-,...,NaN,False,0.0,NaN,True,True,73:33,ACH-003000_g.chr19:4817288_4817289insAGG,ACH-003000_19_4817288,other non-conserving


### check important mutations

In [ ]:
# check MOLM13, MV411 cell lines- The well known mutation status of FLT3

In [ ]:
# check TP53 mutation 

Are mutation consistent?

QC mutations, for a known dependency, check if it matches mutation of this gene. (if P53 is mutated, cannot have dependency on P53 or MDM2 MDM4/ inverse fir BRAF and KRAF to themselves)

### saving this version

In [19]:
merged[merged.DepMap_ID=="ACH-002055"]

,Unnamed: 0,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,...,ExAC_AF,PASS,is_likely_immortalization,CGA_WES_AC,HC_AC,Variant_annotation,RD_AC,RNAseq_AC,SangerWES_AC,WGS_AC
1173867,1070332,ATAD3B,83858,37,1,1425664,1425664,+,Silent,SNP,...,0.000008,True,False,130:79,NaN,silent,NaN,NaN,NaN,NaN
1173868,1070333,SLC45A1,50651,37,1,8390777,8390777,+,Silent,SNP,...,0.000025,True,False,99:172,NaN,silent,NaN,NaN,NaN,NaN
1173869,1070334,PRDM2,7799,37,1,14075946,14075946,+,Missense_Mutation,SNP,...,0.000050,True,False,285:148,NaN,other non-conserving,NaN,NaN,NaN,NaN
1173870,1070335,SPATA21,374955,37,1,16736133,16736133,+,Nonsense_Mutation,SNP,...,0.000008,True,False,125:166,NaN,damaging,NaN,NaN,NaN,NaN
1173871,1070336,CROCC,9696,37,1,17256453,17256453,+,Missense_Mutation,SNP,...,NaN,True,False,30:132,NaN,other non-conserving,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174249,1070686,ESX1,80712,37,X,103495101,103495101,+,Silent,SNP,...,NaN,True,False,49:2,NaN,silent,NaN,NaN,NaN,NaN
1174250,1070687,KIAA1210,57481,37,X,118223032,118223032,+,Missense_Mutation,SNP,...,0.000008,True,False,65:0,NaN,other non-conserving,NaN,NaN,NaN,NaN
1174251,1070688,NKRF,55922,37,X,118724004,118724004,+,Missense_Mutation,SNP,...,NaN,True,False,82:0,NaN,other non-conserving,NaN,NaN,NaN,NaN
1174252,1070689,DCAF12L1,139170,37,X,125686287,125686287,+,Missense_Mutation,SNP,...,NaN,True,False,105:0,NaN,other non-conserving,NaN,NaN,NaN,NaN


In [ ]:
merged.to_csv('temp/wes_somatic_mutations_withlegacy_'+samplesetname+'.csv', index=False)

In [ ]:
merged = pd.read_csv('temp/wes_somatic_mutations_withlegacy_'+samplesetname+'.csv')

In [134]:
mafToMat(merged[merged.Variant_annotation=="damaging"], boolify=True).astype(int).T.to_csv('temp/wes_somatic_mutations_boolmatrix_fordepmap_damaging_'+samplesetname+".csv")
mafToMat(merged[merged.Variant_annotation=="other conserving"], boolify=True).astype(int).T.to_csv('temp/wes_somatic_mutations_boolmatrix_fordepmap_othercons_'+samplesetname+".csv")
mafToMat(merged[merged.Variant_annotation=="other non-conserving"], boolify=True).astype(int).T.to_csv('temp/wes_somatic_mutations_boolmatrix_fordepmap_othernoncons_'+samplesetname+".csv")
mafToMat(merged[(merged.isCOSMIChotspot | merged.isTCGAhotspot)], boolify=True).astype(int).T.to_csv('temp/wes_somatic_mutations_boolmatrix_fordepmap_hotspot_'+samplesetname+'.csv')

### saving samples used for this release

should be the same as in CN otherwise need to do something more complex

In [347]:
#ccle_refsamples.loc[renaming.keys(),samplesetname]=1
#ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.arxspan_id.isin(toadd) & ccle_refsamples.datatype=="wgs"].index,samplesetname]=1
#ccle_refsamples.to_csv('temp/newrefWES.csv')

## retrieving unfiltered mutations

In [135]:
unfiltered = res['unfiltered_CGA_MAF_aggregated']
! gsutil cp $unfiltered "temp/wes_mutation_unfiltered_terra_merged.txt"

NameError: name 'res' is not defined

In [140]:
unfiltered = pd.read_csv('temp/wes_mutation_unfiltered_terra_merged.txt', sep='\t', encoding='L6',na_values=["__UNKNOWN__",'.'], engine='c', dtype=str)

In [141]:
unfiltered['somatic'] = unfiltered['somatic'].replace('nan','False')
unfiltered['HGNC_Status'] = unfiltered['HGNC_Status'].replace('nan','Unapproved')
unfiltered['judgement'] = unfiltered['judgement'].replace('nan','REMOVE')
unfiltered = unfiltered.rename(columns={"i_ExAC_AF":"ExAC_AF","Tumor_Sample_Barcode":'DepMap_ID',"Tumor_Seq_Allele2":"Tumor_Allele"}).drop(columns=['Tumor_Seq_Allele1'])
unfiltered['CGA_WES_AC'] = [str(i[0]) + ':' + str(i[1]) for i in np.nan_to_num(unfiltered[['t_alt_count','t_ref_count']].values.astype(float),0).astype(int)]

ValueError: cannot convert float NaN to integer

In [150]:
toremove = []
subunfilt = unfiltered.iloc[:10000]
for i, val in enumerate(unfiltered.columns):
    h.showcount(i,len(unfiltered.columns))
    if len(set(subunfilt[val])-set(['nan']))==1:
        if len(set(unfiltered[val])-set(['nan']))==1:
            toremove.append(val)
unfiltered = unfiltered.drop(columns=set(toremove))

In [152]:
toint =  ["Start_position", "End_position"]
for val in toint:
    unfiltered[val]  = unfiltered[val].astype(int)

In [153]:
unfiltered.to_csv('temp/mutation_somatic_unfiltered_withreplicates.csv.gz', index=False)

In [154]:
unfiltered = unfiltered[unfiltered.DepMap_ID.isin(renaming.keys())]
unfiltered['DepMap_ID'] = unfiltered['DepMap_ID'].replace(renaming)

/home/jeremie/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [155]:
unfiltered.to_csv('temp/mutation_somatic_unfiltered_all.csv.gz', index=False)

In [156]:
del unfiltered

# uploading on taiga

## Saving to latest version

In [163]:
!gunzip temp/wes_mutation_somatic_unfiltered_withreplicates.csv.gz
!gunzip temp/wes_mutation_somatic_unfiltered_all.csv.gz

In [13]:
tc.update_dataset(dataset_permaname="mutations-latest-ed72",
                 upload_file_path_dict={
        'temp/wes_somatic_mutations_withduplicates_'+samplesetname+'.csv': 'TableCSV',
        'temp/wgs_somatic_mutations_'+samplesetname+'.csv': 'TableCSV',
        'temp/wes_somatic_mutations_deleterious_matrix.csv': 'NumericMatrixCSV',
        'temp/wes_somatic_mutations_other_matrix.csv': 'NumericMatrixCSV',
        'temp/wes_somatic_mutations_hotspot_matrix.csv': 'NumericMatrixCSV',
        'temp/wes_somatic_mutations_all_'+samplesetname+'.csv': 'TableCSV',
        'temp/wes_somatic_mutations_deleterious_boolmatrix.csv': 'NumericMatrixCSV',
        'temp/wes_somatic_mutations_other_boolmatrix.csv': 'NumericMatrixCSV',
        'temp/wes_somatic_mutations_hotspot_boolmatrix.csv': 'NumericMatrixCSV',
        'temp/wes_mutation_somatic_unfiltered_all.csv': 'TableCSV',
        'temp/wes_mutation_somatic_unfiltered_withreplicates.csv': 'TableCSV',
        }, 
                 dataset_description="""
# Mutations

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal.

/!\ This is the most up to date version of the CCLE Mutatios data.
The data is most likely of a better quality that what is on other folder. It is however in beta version as not all changes have either been confirmed or accepted by the DepMap Ops and the DepMap Portal Team.

#Version:

v1: first version from 20Q4. with new ways to compute the binary matrices (a binary and continuous version), version with and without duplicates. the unfiltered list of mutations with all available annotations from the CGA pipeline.


# Notations:

all: every cell lines we have

WES: all data comes from the WExomeS samples we posses

WGS: all data comes from the WGenomeS samples we posses

withreplicates: if we have two different sequencing from a sample, we kept both, see the depmap sample tracker for annotations [https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE](https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE). this dataset is more geared toward QC or in-depth analysis of a particular cell line.

merged: everything from both WGS and WES

latest: only the latest sequencing versions of the samples were kept

genes (gene rpkm):
__Rows__:
__Columns__:
Counts (gene counts):
__Rows__:
__Columns__:
Gene level CN data:
__Rows__:
__Columns__:
 DepMap cell line IDs
 gene names in the format HGNC\_symbol (Entrez\_ID)
DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean
 """)

Uploading wes_somatic_mutations_withduplicates_20Q4...
hitting https://cds.team/taiga/api/datafile/65bcc801912849c2bcb3bb8195c63d54
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: wes_somatic_mutations_w

	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: wes_somatic_mutations_all_20Q4 properly converted and uploaded
Uploading wes_somatic_mutations_deleterious_boolmatrix...
hitting https://cds.team/taiga/api/datafile/65bcc801912849c2bcb3bb8195c63d54
Conversion and upload...:
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Conversio

	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploadin

	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploadin

	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploadin

...

	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploadin

	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploadin

	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploadin

	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploadin

'3433e55473564d8584f4e2970fe2cf97'